In [1]:
%load_ext autoreload
%autoreload 2

In [3]:
import sys, os
module_path = ".."
sys.path.append(os.path.abspath(module_path))

In [5]:
import json
import boto3
from pprint import pprint
from termcolor import colored
from utils import bedrock, print_ww
from utils.bedrock import bedrock_info

os.environ["AWS_DEFAULT_REGION"] = "us-east-1"

boto3_bedrock = bedrock.get_bedrock_client(
  assumed_role=os.environ.get("BEDROCK_ASSUME_ROLE", None),
  endpoint_url=os.environ.get("BEDROCK_ENDPOINT_URL", None),
  region=os.environ.get("AWS_DEFAULT_REGION", None)
)

print(colored("\n== FM Lists ==","green"))
pprint(bedrock_info.get_list_fm_models())

Create new client
  Using region: us-east-1
  Using profile: None
boto3 Bedrock client successfully created!
bedrock-runtime(https://bedrock-runtime.us-east-1.amazonaws.com)

== FM Lists ==
{'Claude-Instant-V1': 'anthropic.claude-instant-v1',
 'Claude-V1': 'anthropic.claude-v1',
 'Claude-V2': 'anthropic.claude-v2',
 'Command': 'cohere.command-text-v14',
 'Jurassic-2-Mid': 'ai21.j2-mid-v1',
 'Jurassic-2-Ultra': 'ai21.j2-ultra-v1',
 'Titan-Embeddings-G1': 'amazon.titan-embed-text-v1',
 'Titan-Text-G1': 'TBD'}


In [8]:
import boto3
import json

modelId = "anthropic.claude-v2"
accept = "application/json"
contentType = "application/json"

prompt = "Hello"
claude_prompt = f"\n\nHuman:{prompt}\n\nAssistant:"
body = json.dumps({
  "prompt": claude_prompt,
  "temperature": 0.1,
  "top_p": 1,
  "top_k": 250,
  "max_tokens_to_sample": 200,
  "stop_sequences": ["\n\nHuman:"]
})

response = boto3_bedrock.invoke_model(body=body, modelId=modelId, accept=accept, contentType=contentType)
response_body = json.loads(response.get('body').read())

print(response_body.get('completion'))

 Hello! Nice to meet you.


### langchain with Bedrock

In [9]:
prompt = "삼성전자의 2022년 매출은 얼마인가요?"
claude_prompt = f"\n\nHuman:{prompt}\n\nAssistant:"

In [11]:
from langchain.llms import Bedrock

textgen_llm = Bedrock(
  model_id=bedrock_info.get_model_id(model_name='Claude-V2'),
  client=boto3_bedrock,
  model_kwargs={
    "prompt": claude_prompt,
    "max_tokens_to_sample": 512,
    "stop_sequences": ["\n\nHuman"],
    "temperature": 0,
    "top_p": 0.999,
  }
)
textgen_llm

Bedrock(client=<botocore.client.BedrockRuntime object at 0x7f9822fbc040>, model_id='anthropic.claude-v2', model_kwargs={'prompt': '\n\nHuman:삼성전자의 2022년 매출은 얼마인가요?\n\nAssistant:', 'max_tokens_to_sample': 512, 'stop_sequences': ['\n\nHuman'], 'temperature': 0, 'top_p': 0.999})

In [12]:
response = textgen_llm(claude_prompt)
print(response)

 삼성전자의 2022년 연간 매출은 아직 정확하게 발표되지 않았습니다. 삼성전자는 매 분기마다 실적발표를 통해 분기별 매출을 공개하고 있습니다. 

2022년도에 대한 삼성전자의 매출 현황은 다음과 같습니다:

- 2022년 1분기 매출: 77조원 
- 2022년 2분기 매출: 77조원
- 2022년 3분기 매출: 76조원

따라서 2022년 1-3분기 누적 매출은 약 230조원입니다. 

삼성전자의 2021년 연간 매출이 279조원이었다는 점을 감안하면, 2022년 연간 매출은 이보다 다소 증가한 280-290조원 수준일 것으로 예상해볼 수 있습니다. 하지만 정확한 2022년 연간 실적은 내년 1월 말에 발표될 2022년 4분기 실적과 함께 알 수 있을 것입니다.


### langchain with BedRock을 Streaming 방식으로

In [13]:
prompt = "삼성전자의 Knox 어카운트 무엇인가요?"
claude_prompt = f"\n\nHuman:{prompt}\n\nAssistant:"

In [19]:
from langchain.llms import Bedrock
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

textgen_lim = Bedrock(
  model_id = bedrock_info.get_model_id(model_name='Claude-V2'),
  client=boto3_bedrock,
  model_kwargs={
    "prompt": claude_prompt,
    "max_tokens_to_sample": 512,
    "stop_sequences": ["\n\nHuman"],
    "temperature": 0,
    "top_p": 0.999,
  },
  streaming=True,
  callbacks=[StreamingStdOutCallbackHandler()]
)
textgen_llm

Bedrock(client=<botocore.client.BedrockRuntime object at 0x7f9822fbc040>, model_id='anthropic.claude-v2', model_kwargs={'prompt': '\n\nHuman:삼성전자의 2022년 매출은 얼마인가요?\n\nAssistant:', 'max_tokens_to_sample': 512, 'stop_sequences': ['\n\nHuman'], 'temperature': 0, 'top_p': 0.999})

In [20]:
num_tokens = textgen_lim.get_num_tokens(prompt)
print(f"Our prompt has {num_tokens} tokens")

Our prompt has 25 tokens


In [21]:
response = textgen_lim(claude_prompt)

 삼성전자의 Knox는 모바일 기기의 보안과 관리를 위한 플랫폼입니다. 

Knox의 주요 기능은 다음과 같습니다:

- Knox Workspace - 업무와 개인 용도로 기기를 분리하여 사용할 수 있습니다. 

- Knox Configure - 기업은 Knox를 통해 기기를 원격으로 설정, 모니터링, 제어할 수 있습니다.

- Knox E-FOTA - 기업은 소프트웨어 업데이트를 원격으로 푸시할 수 있습니다.

- Knox Platform for Enterprise - 기업 앱과 데이터를 보호하기 위한 보안 플랫폼입니다.

Knox 어카운트는 Knox 기능을 사용하기 위해 필요한 삼성 계정입니다. 개인용과 업무용으로 구분되며, 업무용 Knox 어카운트는 기업 관리자가 부여합니다. Knox 어카운트를 통해 기기와 데이터를 안전하게 관리할 수 있습니다.

In [22]:
print(response)

 삼성전자의 Knox는 모바일 기기의 보안과 관리를 위한 플랫폼입니다. 

Knox의 주요 기능은 다음과 같습니다:

- Knox Workspace - 업무와 개인 용도로 기기를 분리하여 사용할 수 있습니다. 

- Knox Configure - 기업은 Knox를 통해 기기를 원격으로 설정, 모니터링, 제어할 수 있습니다.

- Knox E-FOTA - 기업은 소프트웨어 업데이트를 원격으로 푸시할 수 있습니다.

- Knox Platform for Enterprise - 기업 앱과 데이터를 보호하기 위한 보안 플랫폼입니다.

Knox 어카운트는 Knox 기능을 사용하기 위해 필요한 삼성 계정입니다. 개인용과 업무용으로 구분되며, 업무용 Knox 어카운트는 기업 관리자가 부여합니다. Knox 어카운트를 통해 기기와 데이터를 안전하게 관리할 수 있습니다.
